❗TODO
---


1. Придумать как угадать сдвиг (фон паддинга???)
2. Подобрать паддинг относительно размеров картинки (судя по всему - размер/8)
3. Обеспечить безопасное сохранение данных (блок перезаписи)
4. Сделать автосоставление модели нейронной сети по массиву



⚙ Настройки
---

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!nvidia-smi

Sun Feb 19 10:56:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:06:00.0 Off |                    0 |
| N/A   36C    P0    34W / 250W |  31639MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#@title Настройки картинок { run: "auto", vertical-output: true }

Dim_XY = 50 #@param {type:"integer"} # Размеры картинок 
Padding = 20 #@param {type:"integer"} # Размер паддинга

pics_folder_Path = "/home/roloroma/PyTorch/pics/"
pics_Name = "lenna" #@param ["lenna", "strelka", "kristall", "tor_vir"]
pics_Path = pics_folder_Path+'/'+pics_Name+'/'


#@markdown 
#@markdown Для разделения мнимой и действ. картинок
Different_ON = False #@param {type:"boolean"}
Original_Is_Real_ON = False #@param {type:"boolean"}
Different_Is_Real_ON = False #@param {type:"boolean"}
if Different_ON:
    imag_pics_Name =  "test" #@param ["test", "kristall", "tor_vir"] {allow-input: true}
    imag_pics_Path = pics_folder_Path+'/'+imag_pics_Name+'/'

#@markdown 
#@markdown Замена картинки на белое изображение
Real_Zeros_ON = False #@param {type:"boolean"}
Imag_Zeros_ON = True #@param {type:"boolean"}

#@markdown 
#@markdown Загрузка собственного изображения
Real_My_ON = False #@param {type:"boolean"}
Imag_My_ON = False #@param {type:"boolean"}

#@markdown 
#@markdown Поменять местами картинки
Real_Imag_Swap_ON = True #@param {type:"boolean"}

In [4]:
#@title Настройки входного датасета { run: "auto", vertical-output: true }
Dataset_Size = 12000 #@param {type:"integer"} # Размер датасета
Batch_Size = 1000 #@param {type:"integer"} # Размер батча

In [5]:
#@title Загрузка/Тренировка/Сохранение модели { run: "auto", vertical-output: true }
Load_ON = False #@param {type:"boolean"}
Train_ON = True #@param {type:"boolean"}
Save_ON = True #@param {type:"boolean"}
postfix = "_Batch" #@param {type:"string"}

model_folder_Path = "/home/roloroma/PyTorch/models/" # Путь до сохраненных моделей
model_filename = '2D Батч Классификатор ' + pics_Name + ' ' + str(Dim_XY)+'x'+str(Dim_XY)+' padding'+str(Padding)+postfix+'.pt'


In [6]:
#@title Параметры обучения { run: "auto", vertical-output: true }
epochs_Num = 150000 #@param {type:"integer"} # Количество эпох
loss_break = 0.05 #@param {type:"number"} # Значение loss для принудительной остановки


🧰 Импорты и утилити функции
---

In [7]:
# !conda install torchinfo
# !conda install torchviz

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, Subset, DataLoader
# from sklearn.metrics import confusion_matrix
import pandas as pd

# from torchinfo import summary
# from torchviz import make_dot

# import torchinfo
# import torchviz

import math
import numpy as np
import itertools

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)

KeyboardInterrupt: 

In [ ]:
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Функция расчета разницы между фазами на кольце [0, 2pi]
def ring_alg(a,b):
    return math.pi - np.abs(np.abs(a-b)-math.pi)

def plot_image(image, axis, logscale=False, colorbar=True, vmin=None, vmax=None, interpolation='nearest', title = '', fontsize = 20, **kwargs):
    """Plot image to axis with optional logscale and colorbar

    Keyword arguments:
    image -- 2D array with data to show
    axis -- matplotlib axis to plot
    logscale -- use logscale
    colorbar -- add colorbar to image
    vmin -- minimal value of colormap
    vmax -- miximum value of colormap
    """
    if vmin is None and logscale:
        vmin = max(1, np.amin(image))
    elif vmin is None:
        vmin = np.amin(image)

    if vmax is None:
        vmax = np.amax(image)

    if logscale:
        norm = colors.LogNorm(vmin=vmin, vmax=vmax)
    else:
        norm = colors.Normalize(vmin=vmin, vmax=vmax)

    im_handle = axis.imshow(image, norm=norm, interpolation=interpolation, **kwargs)
    axis.set_xticks([])
    axis.set_yticks([])

    axis.set_title(title, fontsize=fontsize)

    if colorbar:
        divider = make_axes_locatable(axis)
        cb_axis = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im_handle, cax=cb_axis)

In [ ]:
def fftn_shift(real_data, dim=None):
    return torch.fft.fftshift(torch.fft.fftn(torch.fft.ifftshift(real_data, dim), dim = dim), dim)

def ifftn_shift(recip_data, dim=None):
    return torch.fft.fftshift(torch.fft.ifftn(torch.fft.ifftshift(recip_data, dim), dim = dim), dim)

def fftn_shift_list(real_data):
    new_recip_data, new_recip_data_nopad, new_real_data = [], [], []
    for (d, l) in real_data:
        d = d.to(device)
        new_real_data.append(d)
        new_recip_data_nopad.append(torch.fft.fftshift(torch.fft.fftn(torch.fft.ifftshift(d))).abs())
        d = torch.nn.functional.pad(d, (Padding, Padding, Padding, Padding))
        new_recip_data.append(torch.fft.fftshift(torch.fft.fftn(torch.fft.ifftshift(d))).abs())
    return torch.stack(new_recip_data), torch.stack(new_recip_data_nopad), torch.stack(new_real_data)

def ifftn_shift_list(recip_data):
    new_real_data, new_recip_data = [], []
    for d in recip_data:
        new_real_data.append(torch.fft.fftshift(torch.fft.ifftn(torch.fft.ifftshift(d)))[Padding:-Padding,Padding:-Padding])
        new_recip_data.append(d)
    return new_real_data, new_recip_data

# def fftn_shift(real_data):
#     return torch.fft.fftn(real_data, norm='ortho')

# def ifftn_shift(recip_data):
#     return torch.fft.ifftn(recip_data, norm='ortho')

🖼 Составляем входные данные
---

Загружаем датасеты и делаем их нужного размера

In [ ]:
# #@title Загружаем датасет двумерных картинок fashion-MNIST и преобразуем его в тензоры размером Dim_XY на Dim_XY
# dataset = torchvision.datasets.FashionMNIST(pics_folder_Path+"/datasets/", download=True, 
#     transform=transforms.Compose([transforms.ToTensor(), transforms.Resize(Dim_XY, interpolation=transforms.InterpolationMode.BILINEAR, max_size=None, antialias='True')]))

In [ ]:
# #@title Загружаем датасет двумерных иероглифов КMNIST и преобразуем его в тензоры размером Dim_XY на Dim_XY
dataset = torchvision.datasets.KMNIST(pics_folder_Path+"/datasets/", download=True, 
    transform=transforms.Compose([transforms.ToTensor(), transforms.Resize(Dim_XY, interpolation=transforms.InterpolationMode.BILINEAR, max_size=None, antialias='True')]))

In [ ]:
#@title Трансформируем данные датасета в необходимый формат: Обратное изображение, обратное изображение без паддинга, прямое изображение с паддингом, прямое изображение.
data = transforms.Resize(Dim_XY, 
                         interpolation=transforms.InterpolationMode.BILINEAR, 
                         max_size=None, antialias='True'
                         )(dataset.data[:Dataset_Size*2]).to(torch.float32)

data = transforms.Normalize((0.5),(255))(data)

data = transforms.RandomRotation(degrees=(0, 180))(data)
data = transforms.RandomVerticalFlip(p=0.5)(data)
data = transforms.GaussianBlur(15, sigma=7)(data)

real_D = torch.nn.functional.pad(data, (Padding,Padding,Padding,Padding))
imag_D = torch.zeros_like(real_D)

full_D = torch.complex(real_D, imag_D)

f_full_D = fftn_shift(full_D, dim=(-2,-1))

f_ampl_D = torch.abs(f_full_D)
f_phase_D = torch.angle(f_full_D)


real_D_nopad = data
imag_D_nopad = torch.zeros_like(real_D_nopad)

full_D_nopad = torch.complex(real_D_nopad, imag_D_nopad)

f_full_D_nopad = fftn_shift(full_D_nopad, dim=(-2,-1))
f_ampl_D_nopad = torch.abs(f_full_D_nopad)
f_phase_D_nopad = torch.angle(f_full_D_nopad)

In [ ]:
#@title Создаем датасет и лоадер

dataset_T = torch.utils.data.TensorDataset(f_ampl_D[:Dataset_Size].unsqueeze(1),f_ampl_D_nopad[:Dataset_Size].unsqueeze(1),real_D[:Dataset_Size].unsqueeze(1),real_D_nopad[:Dataset_Size].unsqueeze(1))
loader_T = torch.utils.data.DataLoader(dataset_T, batch_size=Batch_Size)

dataset_Check = torch.utils.data.TensorDataset(f_ampl_D[Dataset_Size:Dataset_Size*2].unsqueeze(1),f_ampl_D_nopad[Dataset_Size:Dataset_Size*2].unsqueeze(1),real_D[Dataset_Size:Dataset_Size*2].unsqueeze(1),real_D_nopad[Dataset_Size:Dataset_Size*2].unsqueeze(1))
loader_Check = torch.utils.data.DataLoader(dataset_Check, batch_size=Batch_Size)

iter_test = iter(loader_T)

In [ ]:
f_ampl_D_Example, f_ampl_D_nopad_Example, real_D_Example, real_D_nopad_Example = next(iter_test)
f_ampl_D_Example.squeeze_(1)
f_ampl_D_nopad_Example.squeeze_(1)
real_D_Example.squeeze_(1)
real_D_nopad_Example.squeeze_(1)
print(f_ampl_D_Example.size())

fig_orig, ax_orig = plt.subplots(2,3,figsize=(14,8))
plot_image(real_D_Example[0].numpy(), ax_orig[0][0], cmap='gray')
plot_image(f_ampl_D_Example[0].abs().numpy(), ax_orig[0][1], cmap='gray')
plot_image(ifftn_shift(f_ampl_D_Example[0]).real.numpy(), ax_orig[0][2], cmap='gray')
plot_image(real_D_nopad_Example[0].numpy(), ax_orig[1][0], cmap='gray')
plot_image(f_ampl_D_nopad_Example[0].abs().numpy(), ax_orig[1][1], cmap='gray')
plot_image(ifftn_shift(f_ampl_D_nopad_Example[0]).real.numpy(), ax_orig[1][2], cmap='gray')

Открываем картинки вещественных и мнимых значений как массивы

---

1. Тк обрабатываем все послойно, то с размерами проблемы нет
2. Паддинг ухудшает сходимость, но дает огромное улучшение результата. Без него не разделяются дейст. и мним. части.

In [ ]:
if Real_Imag_Swap_ON:
    real_Arr = np.asarray(Image.open(pics_Path+'imag.png').resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))
    # imag_Arr = np.asarray(Image.open(pics_Path+'real.png').resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))
else:
    real_Arr = np.asarray(Image.open(pics_Path+'real.png').resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))
    # imag_Arr = np.asarray(Image.open(pics_Path+'imag.png').resize((Dim_XY, Dim_XY), Image.ANTIALIAS).convert('L'))


# Нормируем картинки
real_Arr = real_Arr/255

# Загружаем разные картинки
if Different_ON:
    if Original_Is_Real_ON:
        real_Arr = np.asarray(Image.open(pics_Path+'real.png').resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))
    else:
        real_Arr = np.asarray(Image.open(pics_Path+'imag.png').resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))

    if Different_Is_Real_ON:
        imag_Arr = np.asarray(Image.open(imag_pics_Path+'real.png').resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))
    else:
        imag_Arr = np.asarray(Image.open(imag_pics_Path+'imag.png').resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))


# Заменяем картинки на белый фон
if Real_Zeros_ON:
    real_Arr = np.zeros((Dim_XY, Dim_XY))
if Imag_Zeros_ON:
    imag_Arr = np.zeros((Dim_XY, Dim_XY))


# Загружаем картинки из локала
if Real_My_ON:
    uploaded = list(files.upload())
    real_Arr = np.asarray(Image.open('/content/'+uploaded[0]).resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))
    rm_str = "'/content/"+uploaded[0]+"'"
    %rm {rm_str}

if Imag_My_ON:
    uploaded = list(files.upload())
    imag_Arr = np.asarray(Image.open('/content/'+uploaded[0]).resize((Dim_XY, Dim_XY), Image.Resampling.LANCZOS).convert('L'))
    rm_str = "'/content/"+uploaded[0]+"'"
    %rm {rm_str}


# Добавляем Паддинг
real_Arr_no_pad = real_Arr.copy()
imag_Arr_no_pad = imag_Arr.copy()

real_Arr = np.pad(real_Arr, Padding)
imag_Arr = np.pad(imag_Arr, Padding)

# Создаем маску паддинга
mask_Arr = np.pad(np.full_like(real_Arr_no_pad, False), Padding, mode='constant', constant_values=True) 


fig_orig, ax_orig = plt.subplots(1,2,figsize=(14,8))

plot_image(real_Arr, ax_orig[0], cmap='gray')
plot_image(imag_Arr, ax_orig[1], cmap='gray')

---
1. Преобразуем данные в тензоры
2. Вычисляем тензор амплитуд и фаз
3. Вычисляем тензор амплитуд и фаз фурье-образа
4. Аналогично делаем для транспонированных картинок

In [ ]:
real_Ten = torch.from_numpy(real_Arr).to(torch.float).to(device) # Скрытая от ИНН информация
imag_Ten = torch.from_numpy(imag_Arr).to(torch.float).to(device) # Скрытая от ИНН информация
mask_Ten = torch.from_numpy(mask_Arr).to(torch.bool).to(device) # Известная ИНН информация 

full_Ten = torch.complex(real_Ten, imag_Ten)
print(full_Ten.dtype)

ampl_Ten = torch.abs(full_Ten) # Известная ИНН информация 
phase_Ten = torch.angle(full_Ten) # Скрытая от ИНН информация
f_ampl_Ten = torch.abs(fftn_shift(full_Ten)) # Известная ИНН информация 
f_phase_Ten = torch.angle(fftn_shift(full_Ten)) # Скрытая от ИНН информация 

fig_ampl, ax_ampl = plt.subplots(1, figsize=(14,8))
plot_image(ampl_Ten.detach().cpu().numpy(), ax_ampl, cmap='gray')

fig_f_ampl, ax_f_ampl = plt.subplots(1, figsize=(14,8))
plot_image(f_ampl_Ten.detach().cpu().numpy(), ax_f_ampl, logscale=True)

fig_f_phase, ax_f_phase = plt.subplots(1, figsize=(14,8))
plot_image(f_phase_Ten.detach().cpu().numpy(), ax_f_phase, cmap='twilight')

In [ ]:
real_Ten_no_pad = torch.from_numpy(real_Arr_no_pad).to(torch.float32).to(device) # Скрытая от ИНН информация
imag_Ten_no_pad = torch.from_numpy(imag_Arr_no_pad).to(torch.float32).to(device) # Скрытая от ИНН информация

full_Ten_no_pad = torch.complex(real_Ten_no_pad, imag_Ten_no_pad)

ampl_Ten_no_pad = torch.abs(full_Ten_no_pad) # Известная ИНН информация 
phase_Ten_no_pad = torch.angle(full_Ten_no_pad) # Скрытая от ИНН информация
f_ampl_Ten_no_pad = torch.abs(fftn_shift(full_Ten_no_pad)) # Известная ИНН информация 
f_phase_Ten_no_pad = torch.angle(fftn_shift(full_Ten_no_pad)) # Скрытая от ИНН информация 

fig_ampl, ax_ampl = plt.subplots(1, figsize=(14,8))
plot_image(ampl_Ten_no_pad.detach().cpu().numpy(), ax_ampl, cmap='gray')

fig_f_ampl, ax_f_ampl = plt.subplots(1, figsize=(14,8))
plot_image(f_ampl_Ten_no_pad.detach().cpu().numpy(), ax_f_ampl, logscale=True)

fig_f_phase, ax_f_phase = plt.subplots(1, figsize=(14,8))
plot_image(f_phase_Ten_no_pad.detach().cpu().numpy(), ax_f_phase, cmap='twilight')

🤖 Создаем версию нейронной сети
---


Нейросеть получает на вход тензор попиксельных амплитуд фурье-образа и его транспонированный вариант, которые должна перевести в тензор попиксельных фаз фурье-образа путем вычисления среднего между обычным и дважды транспонированными тензорами.

---

1. Сигмоида перед последним слоем абсолютно необходима. 

In [ ]:
Dim_XY_P = Dim_XY+Padding*2

class PhaseSolver(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.batch_size = Batch_Size
        self.pic_num = 10
        
        self.c1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 4 * self.pic_num, kernel_size=3, stride=1, padding='same'),
            # nn.BatchNorm2d(num_features=4),
            torch.nn.ReLU(),
            # torch.nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.1),
            )
        
        self.c2 = torch.nn.Sequential(
            torch.nn.Conv2d(4 * self.pic_num, 16 * self.pic_num, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.1),
            )
        
        self.l1 = torch.nn.LazyLinear(int(1*Dim_XY_P**2))
        self.r1 = torch.nn.ReLU()
        
        self.l_d1 = torch.nn.Dropout(0.25)
        
        self.lp = torch.nn.LazyLinear(int(1*Dim_XY_P**2))
        self.sp = torch.nn.Sigmoid()
        
        self.l_pics = torch.nn.LazyLinear(Dim_XY_P**2)
        
        self.klass_c1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 4, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.1),
            )
        
        self.klass_c2 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 16, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.1),
            )
        
        self.klass_l1 = torch.nn.LazyLinear(Dim_XY_P)
        self.klass_r1 = torch.nn.ReLU()
        
        self.klass_l_d1 = torch.nn.Dropout(0.25)
        
        self.klass_lp = torch.nn.LazyLinear(self.pic_num)
        self.klass_Softp = torch.nn.Softmax(dim = -1)
        

    def print_filters(self, n,x):
        n = math.ceil(math.sqrt(n))
        fig, ax = plt.subplots(n,n, figsize=(8,7))
        for i in range(n):
            for j in range(n):
                plot_image(x[n*i+j], ax[i][j], cmap='gray', title=str(n*i+j+1), fontsize=8, logscale=True)
        plt.show()
    
    def forward(self, x, filter_pic_fl = 0):
        
        x = self.c1(x)
        x = self.c2(x)
        
        x = x.view(self.batch_size, self.pic_num, -1)
        
        x = self.l1(x)
        x = self.r1(x)
        
        x = self.l_d1(x)

        x = self.lp(x)
        x = self.sp(x)

        x = self.l_pics(x)
        
        x_pics = x.view(self.batch_size, self.pic_num, Dim_XY_P, Dim_XY_P)
        x = x.view(self.batch_size * self.pic_num, 1, Dim_XY_P, Dim_XY_P)
        
        # Классифицируем картинки
        x = self.klass_c1(x)
        x = self.klass_c2(x)
        
        x = x.view(self.batch_size, -1)
        
        x = self.klass_l1(x)
        x = self.klass_r1(x)
        
        x = self.klass_l_d1(x)

        x = self.klass_lp(x)
        x = self.klass_Softp(x)

        x = torch.sum(x.view(self.batch_size,self.pic_num, 1, 1) * x_pics, dim=-3).view(self.batch_size, 1, Dim_XY_P, Dim_XY_P)
        
        return x
    
    
PS = PhaseSolver().to(device)
PS.train()

Загружаем модель

In [ ]:
# Load_ON = True
# Load_ON = False
if Load_ON:
    PS = torch.load(model_folder_Path+model_filename, map_location=torch.device(device))

Инициализируем модель

In [ ]:
import datetime
PS.train()
batch_iter = next(iter(loader_T)) 
time = datetime.datetime.now()
test_res = PS(batch_iter[0].to(device))
print(datetime.datetime.now()-time)
print(test_res.size())
# make_dot(test_res, params=dict(list(PS.named_parameters()))).render("/content/rnn_torchviz", format="png")

Проверяем размеры модели

In [ ]:
!nvidia-smi
# summary(PS, batch_iter[0].size())

🏃Составляем оптимизатор и функцию ошибки. Запускаем цикл обучения.
---


Функция Ошибки:
1. Получаем на вход угаданный тензор фаз фурье-образа, известный тензор амплитуд фурье-образа и известный тензор амплитуд картинки.
2. Делаем попиксельное вычисление фурье-образа путем умножения амплитуды на косинус+синус*i фазы.
3. Вычисляем угаданную картинку обратным преобразованием фурье.
4. Возвращаем сумму квадратов разниц между значениями угаданой и известной амплитудой картинок как Ошибку.

---

1. AdamW - лушчий. Adam - чуть хуже, NAdam хуже, SGD не сходится.
2. lr можно понизить на порядок когда loss начинает "подпрыгивать".

---
1. Результат становится "хорошим" при loss = 5 (+-1 для разных картинок)
2. Отличный результат при loss < 1

In [ ]:
import datetime
Train_Sum_Time = 0
last_loss = float('inf')
last_lr = 0.0001

optimizer = torch.optim.AdamW(PS.parameters(), lr=last_lr)
# optimizer = torch.optim.Adam(PS.parameters(), lr=0.00001)
# optimizer = torch.optim.NAdam(PS.parameters(), lr=0.0002)
# optimizer = torch.optim.SGD(PS.parameters(), lr=lr, momentum = 0.2)


MSE = nn.MSELoss(reduction='mean')
def PS_Loss(f_phas_Ten_Sum, f_ampl_Ten, f_ampl_Ten_no_pad, cheat_ampl_Ten, mask_Ten = mask_Ten):
    
    
    #==========================================================================================
    # !!!!!!!!!!!!!!!!!! TODO: У ДЕЙСТВИТЕЛЬНОЙ ЧАСТИ ТОЛЬКО ПОЛОЖИТЕЛЬНЫЕ ВЕЛИЧИНЫ !!!!!!!!!!!
    # !!!!!!!!!!!!!!!!!! TODO: ПЕРЕБОР В СЕТИ ПО ПРОМЕЖУТКУ [0;2pi] !!!!!!!!!!!
    #==========================================================================================
    ANN_f_result_Ten = f_ampl_Ten * (torch.cos(f_phas_Ten_Sum) + torch.sin(f_phas_Ten_Sum)*1j)
    
    ANN_comp_Ten = ifftn_shift(ANN_f_result_Ten, dim=(-2,-1))
    ANN_comp_Ten_no_pad = ANN_comp_Ten[:, :, Padding:-Padding, Padding:-Padding]
    ANN_f_ampl_Ten_no_pad = torch.abs(fftn_shift(ANN_comp_Ten_no_pad, dim=(-2,-1)))
    

    loss_phase = MSE(ANN_f_ampl_Ten_no_pad, f_ampl_Ten_no_pad.abs())
    loss_d_phase = MSE(torch.angle(ANN_comp_Ten_no_pad), torch.zeros_like(ANN_comp_Ten_no_pad.real))  #*10
    loss_imag = MSE(ANN_comp_Ten_no_pad.imag, torch.zeros_like(ANN_comp_Ten_no_pad.imag))
    loss_pad = MSE(ANN_comp_Ten.real[:,:,mask_Ten], torch.zeros_like(ANN_comp_Ten.real[:,:,mask_Ten])) *1000 *10
    loss_cheat = MSE(ANN_comp_Ten_no_pad.abs()*255,cheat_ampl_Ten*255)

    loss = loss_cheat + loss_pad + loss_d_phase

    return loss, loss_cheat, loss_pad, loss_d_phase

def train_model(epochs_Num):
    global Train_Sum_Time
    global optimizer
    global last_loss
    global last_lr
    lr_cnt = 0

    for t in range(epochs_Num+1):  
        Epoch_Time = datetime.datetime.now()
        loss, loss_cheat, loss_pad, loss_d_phas = 0, 0, 0, 0
        for f_images, f_images_nopad, cheat_images, cheat_images_nopad in loader_T:
            # Перекладываем тензоры в GPU
            f_images, f_images_nopad, cheat_images, cheat_images_nopad = (f_images.to(device), 
                                                                          f_images_nopad.to(device), 
                                                                          cheat_images.to(device), 
                                                                          cheat_images_nopad.to(device))

            out_train = PS(f_images)

            loss, loss_cheat, loss_pad, loss_d_phas = PS_Loss(out_train, f_images, f_images_nopad, cheat_images_nopad)
            # loss, loss_phase, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
            # loss, loss_phase, loss_imag = PS_Loss(f_phas_Ten)
            # loss, loss_imag, loss_pad = PS_Loss(f_phas_Ten)
            # loss = PS_Loss(f_phas_Ten)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        delta_Time = datetime.datetime.now() - Epoch_Time
        Train_Sum_Time += (datetime.datetime.now() - Epoch_Time).total_seconds()
        
      # if t % (epochs_Num/1000) == 0:
        if True:
            if Save_ON:
                if t % (epochs_Num/50) == 0:
                    torch.save(PS, model_folder_Path+model_filename)
            print('%-4s' % (str(int(t/epochs_Num*1000))+'‰'),
                '- Ep: '+ '%-5s'%str(t), 
                '- LSum: '+'%10s'%('%.6f'%loss.item()), 
                # '- LPha: '+'%10s'%('%.6f'%loss_phase.item()), 
                # '- LIma: '+'%10s'%('%.6f'%loss_imag.item()), 
                '- LD_CHEAT: '+'%10s'%('%.6f'%loss_cheat.item()), 
                '- LDPhas: '+'%10s'%('%.6f'%loss_d_phas.item()), 
                '- LPad: '+'%10s'%('%.6f'%loss_pad.item()), 
                '- Sec: '+'%6s'%'%.2f'%round(Train_Sum_Time,2), 
                '- Log10(Lr): ' + '%2s' % str(int(-math.log10(last_lr))), 
                '- Mem: ' + str(torch.cuda.memory_allocated('cuda')))

        if loss.item() <= loss_break:
            break

        if last_loss < loss.item():
            lr_cnt += 1
            if lr_cnt >= 5:
                lr_cnt = 0
                last_lr = last_lr/10
                optimizer.zero_grad()
                optimizer = torch.optim.AdamW(PS.parameters(), lr=last_lr)
                optimizer.zero_grad()
                last_loss = float('inf')
            else:
                lr_cnt = 0
        last_loss = loss.item()

In [ ]:
if Train_ON:
    PS.train()
    Data_Str = '\nКартинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY)+'x'+str(Dim_XY) + '\nПаддинг: ' + str(Padding)
    print(Data_Str)
    # !nvidia-smi
    train_model(epochs_Num)

In [ ]:
PS(f_ampl_Ten)

🎁Сохраняем модель
---

In [ ]:
if Save_ON:
    torch.save(PS, model_folder_Path+model_filename)

⚛ Рисуем получившиеся картинки
---

Угаданные действ. и мним. части изображения перед выводом отдельно abs().

---
1. Сдвиг фазы обязательно нужно подобрать, иначе неверно считаются дельты.
2. Порядок вывода: Угаданное, Оригинал, Дельта
3. Нажатие на картинку ее увеличивает, но проще открыть в отдельной вкладке.

In [ ]:
#@title Настройки вывода
    
def init_graphs(phase_shift, f_ampl_Ten=f_ampl_Ten.detach().cpu(), phase_Ten_no_pad=phase_Ten_no_pad.detach().cpu(), real_Ten=real_Ten_no_pad.detach().cpu(), imag_Ten=imag_Ten_no_pad.abs().detach().cpu()):
    global ANN_real_Arr
    global ANN_imag_Arr
    global ANN_phase_Arr
    global ANN_f_phase_Arr
    global nopad_real_Arr
    global nopad_imag_Arr
    global nopad_phase_Arr
    global nopad_f_phase_Arr

  

    # Сдвиг фазы для верного сопостовления действительной и мнимой частей
    #@markdown Сдвиг фазы происходит на pi/2 * phase_shift
    # Подготавливаем данные
    f_ampl_Ten = f_ampl_Ten.view(1, 1, Dim_XY_P,Dim_XY_P)

    PS.eval()
    ANN_phase_res = PS(f_ampl_Ten.to(device))+math.pi/2*phase_shift
    PS.train()
    ANN_phase_res = ANN_phase_res.detach().cpu()
    ANN_f_result_Ten = f_ampl_Ten*(torch.cos(ANN_phase_res) + torch.sin(ANN_phase_res)*1j)
    ANN_result_Ten = ifftn_shift(ANN_f_result_Ten, dim=(-2,-1))
    if Padding > 0: # Убираем паддинг
        ANN_result_Ten = ANN_result_Ten[:,:,Padding:-Padding, Padding:-Padding]

    ANN_real_Ten = ANN_result_Ten.real.cpu().detach()
    ANN_imag_Ten = ANN_result_Ten.imag.cpu().detach()
    ANN_phase_Ten = torch.angle(ANN_result_Ten).cpu().detach()
    # ANN_real_Ten = torch.nn.functional.interpolate(ANN_real_Ten.unsqueeze(0).unsqueeze(0), scale_factor=(Dim_XY/ANN_real_Ten.size(0),Dim_XY/ANN_real_Ten.size(1)), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
    # ANN_imag_Ten = torch.nn.functional.interpolate(ANN_imag_Ten.unsqueeze(0).unsqueeze(0), scale_factor=(Dim_XY/ANN_imag_Ten.size(0),Dim_XY/ANN_imag_Ten.size(1)), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)


    ANN_real_Arr = ANN_real_Ten.view(Dim_XY,Dim_XY).numpy()
    ANN_imag_Arr = ANN_imag_Ten.view(Dim_XY,Dim_XY).numpy()
    ANN_phase_Arr = ANN_phase_Ten.view(Dim_XY,Dim_XY).numpy()
    ANN_phase_Arr[ANN_real_Arr <= 0.01] = 0

    ANN_f_phase_Arr = torch.angle(ANN_f_result_Ten).cpu().detach().numpy()

    real_Arr = real_Ten.view(Dim_XY,Dim_XY).detach().cpu().numpy()
    imag_Arr = imag_Ten.view(Dim_XY,Dim_XY).detach().cpu().numpy()
    nopad_phase_Arr = torch.angle(torch.complex(real_Ten,imag_Ten)).view(Dim_XY,Dim_XY).cpu().detach().numpy()
    nopad_real_Arr = real_Arr
    nopad_imag_Arr = imag_Arr

    ANN_f_all_Ten = fftn_shift(torch.from_numpy(ANN_real_Arr + ANN_imag_Arr*1j))
    ANN_f_phase_Arr = torch.angle(ANN_f_all_Ten).numpy()

    nopad_f_phase_Arr = torch.angle(fftn_shift(torch.complex(real_Ten,imag_Ten).view(Dim_XY,Dim_XY))).cpu().detach().numpy()

In [ ]:
Delta_Range = ()
Delta_Range = list(Delta_Range)
    
def graph_print(pic = None, pic_phase_nopad = None, pic_real = None, pic_imag = None, iter_num = 0, phase_print=False):
    right_phase_shift = 0
    min_mean = float('inf')
    
    for phase_shift in range(4):
        if pic == None:
          init_graphs(phase_shift)
        else:
          init_graphs(phase_shift, pic, pic_phase_nopad, pic_real, pic_imag)

        mean = np.mean(ring_alg(ANN_f_phase_Arr, nopad_f_phase_Arr)).item(0)/math.pi*100
        if min_mean > mean:
            right_phase_shift = phase_shift
            min_mean = mean

    if pic == None:
        init_graphs(right_phase_shift)
    else:
        init_graphs(right_phase_shift, pic, pic_phase_nopad, pic_real, pic_imag)
        
    # # Выводим параметры
    # Data_Str = 'Картинки: ' + str(pics_Name) + '\nРазмер: ' + str(Dim_XY) + '\nПаддинг: ' + str(Padding) + '\nИттерация: ' + str(iter_num)
    # Data_fig, Data_ax = plt.subplots(figsize=(28,4))
    # Data_ax.get_xaxis().set_visible(False)
    # Data_ax.get_yaxis().set_visible(False)
    # Data_fig.text(0.2, 0.5, Data_Str, ha='left', va='center', size=45)
    # plt.show()

    # Выводим действ. картинки
    ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,8))
    ANN_fig_real.suptitle('Real part', fontsize=24)

    plot_image(np.abs(ANN_real_Arr), ANN_ax_real[0], cmap='gray', title="Result", vmin=0, vmax=1)
    plot_image(nopad_real_Arr, ANN_ax_real[1], cmap='gray', title="Original", vmin=0, vmax=1)
    plot_image(np.abs(np.abs(ANN_real_Arr)-nopad_real_Arr), ANN_ax_real[2], cmap='gray', title="Delta", vmin=0, vmax=1)
    plt.show()

#     # Выводим "отбивку"
#     Data_fig, Data_ax = plt.subplots(figsize=(28,0.5))
#     Data_ax.get_xaxis().set_visible(False)
#     Data_ax.get_yaxis().set_visible(False)
#     plt.show()

    # # Выводим мним. картинки
    # ANN_fig_imag, ANN_ax_imag = plt.subplots(1,3, figsize=(28,8))
    # ANN_fig_imag.suptitle('Imaginary part', fontsize=24)
    # plot_image(np.abs(ANN_imag_Arr), ANN_ax_imag[0], cmap='gray', title="Result", vmin=0, vmax=1)
    # plot_image(nopad_imag_Arr, ANN_ax_imag[1], cmap='gray', title="Original", vmin=0, vmax=1)
    # plot_image(np.abs(np.abs(ANN_imag_Arr)-nopad_imag_Arr), ANN_ax_imag[2], cmap='gray', title="Delta", vmin=0, vmax=1)

    # Считаем ошибку по фазе обратного пространства
    Delta_Range.append(round(np.mean(ring_alg(ANN_f_phase_Arr, nopad_f_phase_Arr)).item(0)/math.pi*100))
    if phase_print:
      # Выводим "отбивку"
      Data_fig, Data_ax = plt.subplots(figsize=(28,0.5))
      Data_ax.get_xaxis().set_visible(False)
      Data_ax.get_yaxis().set_visible(False)
      plt.show()


      # Выводим фазы фурье-образа
      ANN_fig_f_phase, ANN_ax_f_phase = plt.subplots(1,3, figsize=(28,8))
      ANN_fig_f_phase.suptitle('Direct Phases', fontsize=24)
      plot_image(ANN_phase_Arr, ANN_ax_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
      plot_image(nopad_phase_Arr, ANN_ax_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
      plot_image(ring_alg(ANN_phase_Arr, nopad_phase_Arr), ANN_ax_f_phase[2], cmap='gray', title="Delta")
      
      # # Выводим "отбивку"
      # Data_fig, Data_ax = plt.subplots(figsize=(28,0.5))
      # Data_ax.get_xaxis().set_visible(False)
      # Data_ax.get_yaxis().set_visible(False)
      # plt.show()


      # Выводим фазы фурье-образа картинок без паддинга
      ANN_fig_f_nopad_phase, ANN_ax_nopad_f_phase = plt.subplots(1,3, figsize=(28,8))
      ANN_fig_f_nopad_phase.suptitle('Fourie Transform Phases No Padding', fontsize=24)
      plot_image(ANN_f_phase_Arr, ANN_ax_nopad_f_phase[0], cmap='twilight', title="Result", vmin=-math.pi, vmax=math.pi)
      plot_image(nopad_f_phase_Arr, ANN_ax_nopad_f_phase[1], cmap='twilight', title="Original", vmin=-math.pi, vmax=math.pi)
      plot_image(ring_alg(ANN_f_phase_Arr, nopad_f_phase_Arr), ANN_ax_nopad_f_phase[2], cmap='gray', title="Delta", vmin=0, vmax=math.pi)


In [ ]:
train_iter = iter(loader_T)

In [ ]:
max_pic = 5
PS.batch_size = 1
i = 0
for pic_epoch in train_iter:
  for pic_batch in range(Batch_Size):
    graph_print(pic_epoch[0][pic_batch], pic_epoch[1][pic_batch], pic_epoch[3][pic_batch], torch.zeros_like(pic_epoch[3][pic_batch]), i)
    i += 1
    if i >= max_pic:
      break
  if i >= max_pic:
      break


In [ ]:
print(Delta_Range)

In [ ]:
check_iter = iter(loader_Check)

In [ ]:
max_pic = 100
i = 0
PS.batch_size = 1
for pic_epoch in check_iter:
  for pic_batch in range(Batch_Size):
    graph_print(pic_epoch[0][pic_batch], pic_epoch[1][pic_batch], pic_epoch[3][pic_batch], torch.zeros_like(pic_epoch[3][pic_batch]), i)
    i += 1
    if i >= max_pic:
      break
  if i >= max_pic:
    break


In [ ]:
graph_print()

In [ ]:
init_graphs(0)

ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,8))
ANN_fig_real.suptitle('Real part 0', fontsize=24)

ANN_real_Arr_export = ANN_real_Arr
ANN_real_Arr_export[ANN_real_Arr_export>1] = 1
ANN_real_Arr_export[ANN_real_Arr_export<0] = 0

plot_image(np.abs(ANN_real_Arr), ANN_ax_real[0], cmap='gray', title="Result", vmin=0, vmax=1)
plot_image(nopad_real_Arr, ANN_ax_real[1], cmap='gray', title="Original", vmin=0, vmax=1)
plot_image(np.abs(np.abs(ANN_real_Arr)-nopad_real_Arr), ANN_ax_real[2], cmap='gray', title="Delta", vmin=0, vmax=1)
plt.show()
Image.fromarray(np.abs(ANN_real_Arr*255).astype(np.uint8)).save("ANN0.png")

#===============================
init_graphs(1)

ANN_real_Arr_export = ANN_real_Arr
ANN_real_Arr_export[ANN_real_Arr_export>1] = 1
ANN_real_Arr_export[ANN_real_Arr_export<0] = 0

ANN_fig_real, ANN_ax_real = plt.subplots(1,3, figsize=(28,8))
ANN_fig_real.suptitle('Real part 1', fontsize=24)

plot_image(np.abs(ANN_real_Arr), ANN_ax_real[0], cmap='gray', title="Result", vmin=0, vmax=1)
plot_image(nopad_real_Arr, ANN_ax_real[1], cmap='gray', title="Original", vmin=0, vmax=1)
plot_image(np.abs(np.abs(ANN_real_Arr)-nopad_real_Arr), ANN_ax_real[2], cmap='gray', title="Delta", vmin=0, vmax=1)
plt.show()
Image.fromarray(np.abs(ANN_real_Arr*255).astype(np.uint8)).save("ANN1.png")
Image.fromarray(np.abs(nopad_real_Arr*255).astype(np.uint8)).save("nopad_real.png")
